In [1]:
# Import
import pandas as pd
import requests
import re
import datetime as dt
from datetime import datetime
import pickle
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def requests_retry_session(
    retries=5,
    backoff_factor=0.3,
    status_forcelist=(500, 502, 504),
    session=None,
):
    session = session or requests.Session()
    retry = Retry(
        total=retries,
        read=retries,
        connect=retries,
        backoff_factor=backoff_factor,
        status_forcelist=status_forcelist,
    )
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

# loading the city code
citycode = pd.read_excel('Book1.xlsx')
citycode = citycode.T
citycode = citycode.reset_index()
citycode = citycode['index']
cityframe = pd.DataFrame(columns=['city', 'code'])
for i in range(0, 438):
    cityframe = cityframe.append({'city': citycode[2 * i], 'code': float(citycode[2 * i + 1])}, ignore_index=True)
cityframe['code'] = round(cityframe['code'])
cityframe['code'] = cityframe['code'].astype('int')
cityframe = cityframe.drop_duplicates(subset='code', keep='first')

#load
with open('mo.txt','rb') as f:
    dout = pickle.load(f)

with open('mi.txt','rb') as f:
    din = pickle.load(f)

# adding new data
lastupdate = list(dout.keys())
lastupdate = [dt.datetime.strptime(d,'%Y%m%d').date() for d in lastupdate]
sday = max(lastupdate)+dt.timedelta(days=1)
eday = dt.date.today()-dt.timedelta(days=1)
nperiod = pd.date_range(start=sday, end=eday)
nperiod = nperiod.strftime('%Y%m%d')

for d in nperiod:
    date = d
    rx_dict = {
        'city_name': re.compile(r'"city_name":"(?P<city_name>.*)","province_name"'),
        'value': re.compile(r'"value":(\d*[.]?\d*)')
    }
    moveout = {}
    movein = {}
    mic = list()
    miv = list()
    moc = list()
    mov = list()

    for i in range(0, cityframe.shape[0]):
        city = cityframe.iloc[i][0]
        code = cityframe.iloc[i][1]
        print(city,i,date)
        ourl = 'http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id=%s&type=move_out&date=%s' % (code, date)
        iurl = 'http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id=%s&type=move_in&date=%s' % (code, date)
        try:
            ocityrank = requests_retry_session().get(ourl)
            icityrank = requests_retry_session().get(iurl)
        except Exception as x:
            print('It failed :(', x.__class__.__name__)
        else:
            print('It worked')
        otemp = ocityrank.text.split('},')
        itemp = icityrank.text.split('},')
        if len(otemp) > 1:
            ocityr = pd.DataFrame(columns=['city', 'value'])
            for s in otemp:
                oa = re.findall(rx_dict['city_name'], s)
                ob = re.findall(rx_dict['value'], s)
                ocityr = ocityr.append({'city': oa[0].encode('utf-8').decode('unicode-escape'), 'value': ob[0]},
                                     ignore_index=True)
            moc.append(city)
            mov.append(ocityr)
        if len(itemp) > 1:
            icityr = pd.DataFrame(columns=['city', 'value'])
            for s in itemp:
                ia = re.findall(rx_dict['city_name'], s)
                ib = re.findall(rx_dict['value'], s)
                icityr = icityr.append({'city': ia[0].encode('utf-8').decode('unicode-escape'), 'value': ib[0]},
                                     ignore_index=True)
            mic.append(city)
            miv.append(icityr)
    for key, value in zip(moc, mov):
        moveout[key] = value
    moveout = dict(zip(moc, mov))
    dout[d] = moveout
    for key, value in zip(mic, miv):
        movein[key] = value
    movein = dict(zip(mic, miv))
    din[d] = movein              

    with open('mo.txt', 'wb') as f:
        pickle.dump(dout, f)

    with open('mi.txt', 'wb') as f:
        pickle.dump(din, f)

    


In [2]:
import copy

# Fixing
def Diff(li1,li2):
    return(list(set(li1) - set(li2)))
day1 = dt.date(2020, 1, 1)
dayend = list(din.keys())[-1]
dayend = datetime.strptime(dayend,'%Y%m%d')
period = pd.date_range(start=day1, end=dayend)
period = period.strftime('%Y%m%d')

tempc = list(cityframe['city'])
oli = list()
ili = list()

dinli = []
doutli = []
ddinli = []
ddoutli = []

for key in din.items():
    dinli.append(list(key[1]))
    doutli.append(list(key[1]))

for i in range(0,len(period)):
    ddinli.append(Diff(tempc,dinli[i]))
    ddoutli.append(Diff(tempc,doutli[i]))

ddinli2 = copy.deepcopy(ddinli)
ddoutli2 = copy.deepcopy(ddoutli)
tries = 50
for i in range(0,len(period)):
    date = period[i]
    rx_dict = {
        'city_name': re.compile(r'"city_name":"(?P<city_name>.*)","province_name"'),
        'value': re.compile(r'"value":(\d*[.]?\d*)'),
        'err_msg': re.compile(r'"errmsg":"(?P<err_msg>.*)",')
    }
    if len(ddoutli[i]) > 0:
        olist = ddoutli[i]
        for j in range(0, len(olist)):
            city = olist[j]
            code = cityframe[cityframe['city']==city]['code']
            ourl = 'http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id=%s&type=move_out&date=%s' % (code, date)
            ocond = 0
            oloop = 1
            while ocond == 0 and oloop <= tries:
                print(city, i, date, "oloop=",oloop)
                try:
                    ocityrank = requests_retry_session().get(ourl)
                except Exception as x:
                    print('It failed :(', x.__class__.__name__)
                else:
                    print('It worked')
                otemp = ocityrank.text.split('},')
                err_msg = re.findall(rx_dict['err_msg'],otemp[0])
                if err_msg[0] == 'id is not valid':
                    ddoutli2[i].remove(city)
                    oloop = tries+1
                elif len(otemp) > 2:
                    ocityr = pd.DataFrame(columns=['city', 'value'])
                    for s in otemp:
                        oa = re.findall(rx_dict['city_name'], s)
                        ob = re.findall(rx_dict['value'], s)
                        ocityr = ocityr.append({'city': oa[0].encode('utf-8').decode('unicode-escape'), 'value': ob[0]},
                                             ignore_index=True)
                    dout[date][city] = ocityr
                    ocond = 1
                else:
                    ocond = 0
                    oloop = oloop+1
            if oloop >= tries:
                oli.append([date,city,err_msg])



    if len(ddinli[i]) > 0:
        ilist = ddinli[i]
        for j in range(0, len(ilist)):
            city = ilist[j]
            code = cityframe[cityframe['city']==city]['code']
            iurl = 'http://huiyan.baidu.com/migration/cityrank.jsonp?dt=province&id=%s&type=move_in&date=%s' % (code, date)
            icond = 0
            iloop = 1
            while icond == 0 and iloop <= tries:
                print(city, i, date, "iloop=",iloop)
                try:
                    icityrank = requests_retry_session().get(iurl)
                except Exception as x:
                    print('It failed :(', x.__class__.__name__)
                else:
                    print('It worked')
                itemp = icityrank.text.split('},')
                err_msg = re.findall(rx_dict['err_msg'],otemp[0])
                if err_msg[0] == 'id is not valid':
                    ddinli2[i].remove(city)
                    iloop = tries+1
                elif len(itemp) > 2:
                    icityr = pd.DataFrame(columns=['city', 'value'])
                    for s in itemp:
                        ia = re.findall(rx_dict['city_name'], s)
                        ib = re.findall(rx_dict['value'], s)
                        icityr = icityr.append({'city': ia[0].encode('utf-8').decode('unicode-escape'), 'value': ib[0]},
                                             ignore_index=True)
                    din[date][city] = icityr
                    icond = 1
                else:
                    icond = 0
                    iloop = iloop + 1
                if iloop >= tries:
                    ili.append([date, city, err_msg])


襄阳 0 20200101 oloop= 1
It worked
西宁 0 20200101 oloop= 1
It worked
黄冈 0 20200101 oloop= 1
It worked
新乡 0 20200101 oloop= 1
It worked
桂林 0 20200101 oloop= 1
It worked
宜宾 0 20200101 oloop= 1
It worked
巢湖 0 20200101 oloop= 1
It worked
孝感 0 20200101 oloop= 1
It worked
柳州 0 20200101 oloop= 1
It worked
襄阳 0 20200101 iloop= 1
It worked
西宁 0 20200101 iloop= 1
It worked
黄冈 0 20200101 iloop= 1
It worked
新乡 0 20200101 iloop= 1
It worked
桂林 0 20200101 iloop= 1
It worked
宜宾 0 20200101 iloop= 1
It worked
巢湖 0 20200101 iloop= 1
It worked
孝感 0 20200101 iloop= 1
It worked
柳州 0 20200101 iloop= 1
It worked
宜宾 1 20200102 oloop= 1
It worked
巢湖 1 20200102 oloop= 1
It worked
襄阳 1 20200102 oloop= 1
It worked
郑州 1 20200102 oloop= 1
It worked
宜宾 1 20200102 iloop= 1
It worked
巢湖 1 20200102 iloop= 1
It worked
襄阳 1 20200102 iloop= 1
It worked
郑州 1 20200102 iloop= 1
It worked
巢湖 2 20200103 oloop= 1
It worked
黄冈 2 20200103 oloop= 1
It worked
襄阳 2 20200103 oloop= 1
It worked
西宁 2 20200103 oloop= 1
It worked
巢湖 2 20200

It worked
石家庄 7 20200108 iloop= 42
It worked
石家庄 7 20200108 iloop= 43
It worked
石家庄 7 20200108 iloop= 44
It worked
石家庄 7 20200108 iloop= 45
It worked
石家庄 7 20200108 iloop= 46
It worked
石家庄 7 20200108 iloop= 47
It worked
石家庄 7 20200108 iloop= 48
It worked
石家庄 7 20200108 iloop= 49
It worked
石家庄 7 20200108 iloop= 50
It worked
郑州 7 20200108 iloop= 1
It worked
郑州 7 20200108 iloop= 2
It worked
郑州 7 20200108 iloop= 3
It worked
郑州 7 20200108 iloop= 4
It worked
郑州 7 20200108 iloop= 5
It worked
郑州 7 20200108 iloop= 6
It worked
郑州 7 20200108 iloop= 7
It worked
郑州 7 20200108 iloop= 8
It worked
郑州 7 20200108 iloop= 9
It worked
郑州 7 20200108 iloop= 10
It worked
郑州 7 20200108 iloop= 11
It worked
郑州 7 20200108 iloop= 12
It worked
郑州 7 20200108 iloop= 13
It worked
郑州 7 20200108 iloop= 14
It worked
郑州 7 20200108 iloop= 15
It worked
郑州 7 20200108 iloop= 16
It worked
郑州 7 20200108 iloop= 17
It worked
郑州 7 20200108 iloop= 18
It worked
郑州 7 20200108 iloop= 19
It worked
郑州 7 20200108 iloop= 20
It worked
郑州 7

It worked
那曲 12 20200113 oloop= 1
It worked
珠海 12 20200113 oloop= 1
It worked
海口 12 20200113 oloop= 1
It worked
湛江 12 20200113 oloop= 1
It worked
曲靖 12 20200113 oloop= 1
It worked
郑州 12 20200113 oloop= 1
It worked
巢湖 12 20200113 oloop= 1
It worked
广元 12 20200113 oloop= 1
It worked
商丘 12 20200113 oloop= 1
It worked
深圳 12 20200113 iloop= 1
It worked
广州 12 20200113 iloop= 1
It worked
三亚.1 12 20200113 iloop= 1
It worked
那曲 12 20200113 iloop= 1
It worked
珠海 12 20200113 iloop= 1
It worked
海口 12 20200113 iloop= 1
It worked
湛江 12 20200113 iloop= 1
It worked
曲靖 12 20200113 iloop= 1
It worked
郑州 12 20200113 iloop= 1
It worked
巢湖 12 20200113 iloop= 1
It worked
广元 12 20200113 iloop= 1
It worked
商丘 12 20200113 iloop= 1
It worked
三亚.1 13 20200114 oloop= 1
It worked
吉安 13 20200114 oloop= 1
It worked
那曲 13 20200114 oloop= 1
It worked
珠海 13 20200114 oloop= 1
It worked
巢湖 13 20200114 oloop= 1
It worked
三亚.1 13 20200114 iloop= 1
It worked
三亚.1 13 20200114 iloop= 2
It worked
三亚.1 13 20200114 iloop= 3
It w

It worked
佛山 18 20200119 oloop= 1
It worked
三亚.1 18 20200119 oloop= 1
It worked
巢湖 18 20200119 oloop= 1
It worked
南阳 18 20200119 oloop= 1
It worked
南宁 18 20200119 oloop= 1
It worked
三亚 18 20200119 iloop= 1
It worked
深圳 18 20200119 iloop= 1
It worked
佛山 18 20200119 iloop= 1
It worked
三亚.1 18 20200119 iloop= 1
It worked
巢湖 18 20200119 iloop= 1
It worked
南阳 18 20200119 iloop= 1
It worked
南宁 18 20200119 iloop= 1
It worked
长沙 19 20200120 oloop= 1
It worked
江门 19 20200120 oloop= 1
It worked
襄阳 19 20200120 oloop= 1
It worked
常德 19 20200120 oloop= 1
It worked
乌鲁木齐 19 20200120 oloop= 1
It worked
延边 19 20200120 oloop= 1
It worked
周口 19 20200120 oloop= 1
It worked
玉溪 19 20200120 oloop= 1
It worked
那曲 19 20200120 oloop= 1
It worked
阳江 19 20200120 oloop= 1
It worked
韶关 19 20200120 oloop= 1
It worked
遵义 19 20200120 oloop= 1
It worked
宜宾 19 20200120 oloop= 1
It worked
巢湖 19 20200120 oloop= 1
It worked
曲靖 19 20200120 oloop= 1
It worked
拉萨 19 20200120 oloop= 1
It worked
长沙 19 20200120 iloop= 1
It worke

It worked
青岛 21 20200122 iloop= 10
It worked
青岛 21 20200122 iloop= 11
It worked
青岛 21 20200122 iloop= 12
It worked
青岛 21 20200122 iloop= 13
It worked
青岛 21 20200122 iloop= 14
It worked
青岛 21 20200122 iloop= 15
It worked
青岛 21 20200122 iloop= 16
It worked
青岛 21 20200122 iloop= 17
It worked
青岛 21 20200122 iloop= 18
It worked
青岛 21 20200122 iloop= 19
It worked
青岛 21 20200122 iloop= 20
It worked
青岛 21 20200122 iloop= 21
It worked
青岛 21 20200122 iloop= 22
It worked
青岛 21 20200122 iloop= 23
It worked
青岛 21 20200122 iloop= 24
It worked
青岛 21 20200122 iloop= 25
It worked
青岛 21 20200122 iloop= 26
It worked
青岛 21 20200122 iloop= 27
It worked
青岛 21 20200122 iloop= 28
It worked
青岛 21 20200122 iloop= 29
It worked
青岛 21 20200122 iloop= 30
It worked
青岛 21 20200122 iloop= 31
It worked
青岛 21 20200122 iloop= 32
It worked
青岛 21 20200122 iloop= 33
It worked
青岛 21 20200122 iloop= 34
It worked
青岛 21 20200122 iloop= 35
It worked
青岛 21 20200122 iloop= 36
It worked
青岛 21 20200122 iloop= 37
It worked
青岛 21 2020

...

It worked
琼中黎族苗族自治县 53 20200223 iloop= 19
It worked
琼中黎族苗族自治县 53 20200223 iloop= 20
It worked
琼中黎族苗族自治县 53 20200223 iloop= 21
It worked
琼中黎族苗族自治县 53 20200223 iloop= 22
It worked
琼中黎族苗族自治县 53 20200223 iloop= 23
It worked
琼中黎族苗族自治县 53 20200223 iloop= 24
It worked
琼中黎族苗族自治县 53 20200223 iloop= 25
It worked
琼中黎族苗族自治县 53 20200223 iloop= 26
It worked
琼中黎族苗族自治县 53 20200223 iloop= 27
It worked
琼中黎族苗族自治县 53 20200223 iloop= 28
It worked
琼中黎族苗族自治县 53 20200223 iloop= 29
It worked
琼中黎族苗族自治县 53 20200223 iloop= 30
It worked
琼中黎族苗族自治县 53 20200223 iloop= 31
It worked
琼中黎族苗族自治县 53 20200223 iloop= 32
It worked
琼中黎族苗族自治县 53 20200223 iloop= 33
It worked
琼中黎族苗族自治县 53 20200223 iloop= 34
It worked
琼中黎族苗族自治县 53 20200223 iloop= 35
It worked
琼中黎族苗族自治县 53 20200223 iloop= 36
It worked
琼中黎族苗族自治县 53 20200223 iloop= 37
It worked
琼中黎族苗族自治县 53 20200223 iloop= 38
It worked
琼中黎族苗族自治县 53 20200223 iloop= 39
It worked
琼中黎族苗族自治县 53 20200223 iloop= 40
It worked
琼中黎族苗族自治县 53 20200223 iloop= 41
It worked
琼中黎族苗族自治县 53 20200223 il

It worked
遵义 53 20200223 iloop= 35
It worked
遵义 53 20200223 iloop= 36
It worked
遵义 53 20200223 iloop= 37
It worked
遵义 53 20200223 iloop= 38
It worked
遵义 53 20200223 iloop= 39
It worked
遵义 53 20200223 iloop= 40
It worked
遵义 53 20200223 iloop= 41
It worked
遵义 53 20200223 iloop= 42
It worked
遵义 53 20200223 iloop= 43
It worked
遵义 53 20200223 iloop= 44
It worked
遵义 53 20200223 iloop= 45
It worked
遵义 53 20200223 iloop= 46
It worked
遵义 53 20200223 iloop= 47
It worked
遵义 53 20200223 iloop= 48
It worked
遵义 53 20200223 iloop= 49
It worked
遵义 53 20200223 iloop= 50
It worked
巢湖 53 20200223 iloop= 1
It worked
克孜勒苏州 53 20200223 iloop= 1
It worked
克孜勒苏州 53 20200223 iloop= 2
It worked
克孜勒苏州 53 20200223 iloop= 3
It worked
克孜勒苏州 53 20200223 iloop= 4
It worked
克孜勒苏州 53 20200223 iloop= 5
It worked
克孜勒苏州 53 20200223 iloop= 6
It worked
克孜勒苏州 53 20200223 iloop= 7
It worked
克孜勒苏州 53 20200223 iloop= 8
It worked
克孜勒苏州 53 20200223 iloop= 9
It worked
克孜勒苏州 53 20200223 iloop= 10
It worked
克孜勒苏州 53 20200223 iloop= 

It worked
乐东黎族自治县 53 20200223 iloop= 11
It worked
乐东黎族自治县 53 20200223 iloop= 12
It worked
乐东黎族自治县 53 20200223 iloop= 13
It worked
乐东黎族自治县 53 20200223 iloop= 14
It worked
乐东黎族自治县 53 20200223 iloop= 15
It worked
乐东黎族自治县 53 20200223 iloop= 16
It worked
乐东黎族自治县 53 20200223 iloop= 17
It worked
乐东黎族自治县 53 20200223 iloop= 18
It worked
乐东黎族自治县 53 20200223 iloop= 19
It worked
乐东黎族自治县 53 20200223 iloop= 20
It worked
乐东黎族自治县 53 20200223 iloop= 21
It worked
乐东黎族自治县 53 20200223 iloop= 22
It worked
乐东黎族自治县 53 20200223 iloop= 23
It worked
乐东黎族自治县 53 20200223 iloop= 24
It worked
乐东黎族自治县 53 20200223 iloop= 25
It worked
乐东黎族自治县 53 20200223 iloop= 26
It worked
乐东黎族自治县 53 20200223 iloop= 27
It worked
乐东黎族自治县 53 20200223 iloop= 28
It worked
乐东黎族自治县 53 20200223 iloop= 29
It worked
乐东黎族自治县 53 20200223 iloop= 30
It worked
乐东黎族自治县 53 20200223 iloop= 31
It worked
乐东黎族自治县 53 20200223 iloop= 32
It worked
乐东黎族自治县 53 20200223 iloop= 33
It worked
乐东黎族自治县 53 20200223 iloop= 34
It worked
乐东黎族自治县 53 20200223 iloop= 35


It worked
玉树州 53 20200223 iloop= 20
It worked
玉树州 53 20200223 iloop= 21
It worked
玉树州 53 20200223 iloop= 22
It worked
玉树州 53 20200223 iloop= 23
It worked
玉树州 53 20200223 iloop= 24
It worked
玉树州 53 20200223 iloop= 25
It worked
玉树州 53 20200223 iloop= 26
It worked
玉树州 53 20200223 iloop= 27
It worked
玉树州 53 20200223 iloop= 28
It worked
玉树州 53 20200223 iloop= 29
It worked
玉树州 53 20200223 iloop= 30
It worked
玉树州 53 20200223 iloop= 31
It worked
玉树州 53 20200223 iloop= 32
It worked
玉树州 53 20200223 iloop= 33
It worked
玉树州 53 20200223 iloop= 34
It worked
玉树州 53 20200223 iloop= 35
It worked
玉树州 53 20200223 iloop= 36
It worked
玉树州 53 20200223 iloop= 37
It worked
玉树州 53 20200223 iloop= 38
It worked
玉树州 53 20200223 iloop= 39
It worked
玉树州 53 20200223 iloop= 40
It worked
玉树州 53 20200223 iloop= 41
It worked
玉树州 53 20200223 iloop= 42
It worked
玉树州 53 20200223 iloop= 43
It worked
玉树州 53 20200223 iloop= 44
It worked
玉树州 53 20200223 iloop= 45
It worked
玉树州 53 20200223 iloop= 46
It worked
玉树州 53 20200223 il

It worked
宜宾 54 20200224 oloop= 1
It worked
海南州 54 20200224 oloop= 1
It worked
甘南州 54 20200224 oloop= 1
It worked
铁门关 54 20200224 oloop= 1
It worked
德宏州 54 20200224 oloop= 1
It worked
昌江黎族自治县 54 20200224 oloop= 1
It worked
金昌 54 20200224 oloop= 1
It worked
平凉 54 20200224 oloop= 1
It worked
达州 54 20200224 oloop= 1
It worked
六盘水 54 20200224 oloop= 1
It worked
大理州 54 20200224 oloop= 1
It worked
丽水 54 20200224 oloop= 1
It worked
丽江 54 20200224 oloop= 1
It worked
嘉峪关 54 20200224 oloop= 1
It worked
乐东黎族自治县 54 20200224 oloop= 1
It worked
武威 54 20200224 oloop= 1
It worked
昆玉 54 20200224 oloop= 1
It worked
保亭黎族苗族自治县 54 20200224 oloop= 1
It worked
西双版纳傣族自治州 54 20200224 oloop= 1
It worked
固原 54 20200224 oloop= 1
It worked
梧州 54 20200224 oloop= 1
It worked
阿坝州 54 20200224 oloop= 1
It worked
汕尾 54 20200224 oloop= 1
It worked
巴音郭楞蒙古自治州 54 20200224 oloop= 1
It worked
博尔塔拉州 54 20200224 oloop= 1
It worked
山南 54 20200224 oloop= 1
It worked
陵水黎族自治县 54 20200224 oloop= 1
It worked
玉溪 54 20200224 oloop= 1
I

In [4]:
with open('mo.txt', 'wb') as f:
    pickle.dump(dout, f)

with open('mi.txt', 'wb') as f:
    pickle.dump(din, f)

#
ddinli = copy.deepcopy(ddinli2)
ddoutli = copy.deepcopy(ddoutli2)

In [6]:
din.keys()

dict_keys(['20200101', '20200102', '20200103', '20200104', '20200105', '20200106', '20200107', '20200108', '20200109', '20200110', '20200111', '20200112', '20200113', '20200114', '20200115', '20200116', '20200117', '20200118', '20200119', '20200120', '20200121', '20200122', '20200123', '20200124', '20200125', '20200126', '20200127', '20200128', '20200129', '20200130', '20200131', '20200201', '20200202', '20200203', '20200204', '20200205', '20200206', '20200207', '20200208', '20200209', '20200210', '20200211', '20200212', '20200213', '20200214', '20200215', '20200216', '20200217', '20200218', '20200219', '20200220', '20200221', '20200222', '20200223', '20200224', '20200225', '20200226', '20200227'])